# Music Genre Classifier


In [ ]:
import os
import cv2
import random
import shutil
import zipfile
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from pandas import DataFrame
from keras import Sequential
from sklearn.model_selection import train_test_split, KFold
from keras.layers import Conv2D, UpSampling2D, MaxPooling2D, Input, Cropping2D, Cropping3D, Flatten, Dense, Reshape

In [ ]:
SEED_VALUE = 42

# Fix seed to make training deterministic.
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
tf.random.set_seed(SEED_VALUE)

## Load the GTZAN Dataset


In [ ]:
if os.getenv('COLAB_RELEASE_TAG'):
    from google.colab import drive
    drive.mount('/content/gdrive')

In [ ]:
PREPROCESSING = False

try:
    if 'dataset' not in os.listdir('/content'):
        os.mkdir('/content/dataset/')
    pwd = os.getcwd()
    os.chdir('/content/dataset/')

    if 'gtzan-features.zip' not in os.listdir('.'):
        if os.getenv('COLAB_RELEASE_TAG'):
            if 'gtzan-features.zip' in os.listdir('/content/gdrive/MyDrive'):
                shutil.copy2('/content/gdrive/MyDrive/gtzan-features.zip', '.')
            else:
                # make sure to download the GTZAN dataset from "https://drive.google.com/file/d/1LDlvwUeJ-h3JntNSmQABNV8z8oqltojO/view?usp=drive_link"
                # upload it to your own google drive for it to be copied in the previous if statement block
                pass
        else:
            if pwd != '/content/dataset':
                if 'gtzan-features.zip' in os.listdir(f'{pwd}/dataset'):
                    shutil.copy2(f'{pwd}/dataset/gtzan-features.zip', '.')
                else:
                    raise Exception("Download the GTZAN dataset preprocessed.")

        with zipfile.ZipFile('gtzan-features.zip', 'r') as zip_ref:
            zip_ref.extractall('.')

finally:
    GENRES = os.listdir('/content/dataset/training/mfcc')


## Dataset


In [ ]:
def load_data(src: str, feature: str, 
              random_state: float = SEED_VALUE, shuffle: bool = True, 
              stratify: list = None):
    dataset = []
    for genre in os.listdir(f'{src}/{feature}'):
        for img in os.listdir(f'{src}/{feature}/{genre}'):
            img = cv2.imread(f'{src}/{feature}/{genre}/{img}')
            img = cv2.resize(img, (256, 192))
            img = np.array(img, dtype=np.float32)
            dataset.append([img, genre])

    df = DataFrame(data=np.array(dataset, dtype=object), columns=[feature, 'genre'])

    one_hot = pd.get_dummies(df['genre'])

    df = pd.concat([df, one_hot], axis=1)
    df.drop(['genre'], axis=1, inplace=True)

    return (np.array([tf.convert_to_tensor(img) for img in df['mfcc']]), df[GENRES])

In [ ]:
(X_train, _) = load_data('./training/', 'mfcc', stratify=GENRES)
(X_test, _) = load_data('./tests/', 'mfcc', stratify=GENRES)
(X_val, _) = load_data('./validation/', 'mfcc', stratify=GENRES)

# for cross validation purposes it is encouraged to concatenate all input/output data, in this case, only input
X_all = np.concatenate((X_train, X_test), axis=0)
X_all = np.concatenate((X_all, X_val), axis=0)
# this is just to normalize the values between 0 and 1
X_train = X_train / 255
X_test = X_test / 255
X_val = X_val / 255
X_all = X_all / 255

## CNN Autoencoder


In [ ]:
def generate_autoencoder():
    input_l=Input(shape=(192, 256, 3))

    maxp_ini=MaxPooling2D((4,4), padding='same', strides=(2, 2))(input_l)
    encoding_1=Conv2D(12, (3,3), activation='relu',padding='same')(maxp_ini)

    maxp_1=MaxPooling2D((2,2), padding='same')(encoding_1)
    encoding_2=Conv2D(6, (3,3), activation='relu',padding='same')(maxp_1)

    maxp_2=MaxPooling2D((2,2), padding='same')(encoding_2)
    encoding_3=Conv2D(3, (3,3), activation='relu',padding='same')(maxp_2)

    flat_1=Flatten()(encoding_3)

    bottleneck=Dense(500)(flat_1)

    decoding_1=Dense(2304, activation='relu')(bottleneck)
    resh_1=Reshape(target_shape=(24, 32, 3)) (decoding_1)

    decoding_2=Conv2D(6, (3,3), activation='relu', padding='same')(resh_1)
    Up_1=UpSampling2D((2,2))(decoding_2)

    decoding_3=Conv2D(12, (3,3), activation='relu', padding='same')(Up_1)
    Up_2=UpSampling2D((2,2))(decoding_3)

    decoding_4=Conv2D(3, (3,3), activation='relu', padding='same')(Up_2)
    output_l=UpSampling2D((2,2))(decoding_4)

    autoencoder=Model(inputs=[input_l],outputs=[output_l])

    autoencoder.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])

    return autoencoder

In [ ]:
generate_autoencoder().summary()

In [ ]:
model_storage = '/content'

if os.getenv('COLAB_RELEASE_TAG'):
    model_storage = '/content/gdrive/MyDrive'

if 'model_storage_final' not in os.listdir(model_storage):
    os.mkdir(f'{model_storage}/model_storage_final')

model_storage='/content/gdrive/MyDrive/model_storage_final'

## Training and Cross Validating

In [ ]:
autoencoder = generate_autoencoder()
last_iter = 0
iter_num = 15

if len([autoencoder for autoencoder in os.listdir(model_storage) if autoencoder[:19]=='autoencoder_model2_']) > 0:
    last_iter = sorted([int(autoencoder.removeprefix(f'autoencoder_model2_').removesuffix('.keras'))
        for autoencoder in os.listdir(model_storage) if autoencoder[:19]=='autoencoder_model2_'], reverse=True)[0]
    autoencoder = tf.keras.saving.load_model(f'{model_storage}/autoencoder_model2_{last_iter}.keras')
    last_iter+=1 #this is for not overwriting the last saved model

for i in range(last_iter, iter_num):
    autoencoder.fit(x=X_train, y=X_train, epochs=10, validation_data=(X_test, X_test))
    autoencoder.save(f'{model_storage}/autoencoder_model2_{i}.keras')

# Generate validation set metrics
scores = autoencoder.evaluate(X_val, X_val, verbose=0)
print(f'Score for validation set: {autoencoder.metrics_names[0]} of {scores[0]}')

In [ ]:
kfold=KFold(n_splits=10, shuffle=True)
kfold_splits=[]

for train, test in kfold.split(X_all, X_all):
    kfold_splits.append((train, test))

In [ ]:
fold_no=1
final_errors_per_fold=[]

for train, test in kfold_splits:
    last_iter = 0

    autoencoder = generate_autoencoder()

    if f'fold_{fold_no}' not in os.listdir(model_storage):
        os.mkdir(f'{model_storage}/fold_{fold_no}')

    if len([autoencoder for autoencoder in os.listdir(f'{model_storage}/fold_{fold_no}') if autoencoder[:19]=='autoencoder_model2_']) > 0:
        last_iter = sorted([int(autoencoder.removeprefix(f'autoencoder_model2_').removesuffix('.keras'))
            for autoencoder in os.listdir(f'{model_storage}/fold_{fold_no}') if autoencoder[:19]=='autoencoder_model2_'], reverse=True)[0]
        autoencoder = tf.keras.saving.load_model(f'{model_storage}/fold_{fold_no}/autoencoder_model2_{last_iter}.keras')
        last_iter+=1 #this is for not overwriting the last saved model

    for i in range(last_iter, iter_num):
        autoencoder.fit(x=X_all[train], y=X_all[train], epochs=10)
        autoencoder.save(f'{model_storage}/fold_{fold_no}/autoencoder_model2_{i}.keras')

    # Generate generalization metrics
    scores = autoencoder.evaluate(X_all[test], X_all[test], verbose=0)
    print(f'Score for fold {fold_no}: {autoencoder.metrics_names[0]} of {scores[0]}')
    final_errors_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1


plt.figure(figsize=(7,3))

plt.plot(final_errors_per_fold, label='fold test set')
plt.title('Loss Function (MSE)')
plt.xlabel('Fold #')
plt.ylabel('Mean Squared Error')
plt.legend()

plt.show()

## Overfitting analysis

In [ ]:
autoencoder = generate_autoencoder()
last_iter = 0
iter_num = 50
history_train=[]
history_test=[]

for i in range(last_iter, iter_num):
    if f'overfitting_analysis_{i}.keras' in [autoencoder for autoencoder in os.listdir(model_storage)]:
        autoencoder = tf.keras.saving.load_model(f'{model_storage}/overfitting_analysis_{i}.keras')
    else:
        autoencoder.fit(x=X_train, y=X_train, epochs=10, validation_data=(X_test, X_test))

    scores_train = autoencoder.evaluate(X_train, X_train, verbose=0)
    history_train.append(scores_train[0])

    scores_test = autoencoder.evaluate(X_test, X_test, verbose=0)
    history_test.append(scores_test[0])

    if f'overfitting_analysis_{i}.keras' not in [autoencoder for autoencoder in os.listdir(model_storage)]:
        autoencoder.save(f'{model_storage}/overfitting_analysis_{i}.keras')

plt.figure(figsize=(7,3))

plt.subplot(1,2,1)
plt.plot(history_train, label='train')
plt.plot(history_test, label='test')
plt.title('Loss Function (MSE)')
plt.xlabel('Save')
plt.ylabel('Mean Squared Error')
plt.legend()

plt.show()

## Encoder

In [ ]:
def build_encoder(autoencoder_name):
    if autoencoder_name not in os.listdir(model_storage):
        print(f'Not finished optimizing: {autoencoder_name} not found in {model_storage}.')
    else:
        autoencoder = tf.keras.saving.load_model(f'{model_storage}/{autoencoder_name}')
        encoder_input_l = Input(shape=(192,256,3))
        res_1 = autoencoder.layers[1](encoder_input_l)
        res_2 = autoencoder.layers[2](res_1)
        res_3 = autoencoder.layers[3](res_2)
        res_4 = autoencoder.layers[4](res_3)
        res_5 = autoencoder.layers[5](res_4)
        res_6 = autoencoder.layers[6](res_5)
        res_7 = autoencoder.layers[7](res_6)
        encoder_output_l = autoencoder.layers[8](res_7)
        encoder=Model(inputs=[encoder_input_l],outputs=[encoder_output_l])
        return encoder

# if necessary, download this precise autoencoder model from https://drive.google.com/file/d/12gtgkBgo_n1SF6LKLzNsXgtTuN_Wcoqn/view?usp=sharing
encoder = build_encoder('autoencoder_model2_14.keras')
# encoder is already saved at https://drive.google.com/file/d/11odRe5Stk9x8BWTv5Y2x8TFvgT_iuuZs/view?usp=sharing
encoder.save(f'{model_storage}/encoder_model.keras')

print(encoder(X_train[:1])) # test